# Imports / Configuration

In [60]:
import numpy as np
import time

import sys,os

data_path = os.getcwd()

try:
    import localgraphclustering as lgc
except:
    # when the package is not installed, import the local version instead. 
    # the notebook must be placed in the original "notebooks/" folder
    sys.path.append("../")
    import localgraphclustering as lgc

## Load dataset

In [70]:
g = lgc.GraphLocal(os.path.join(data_path,'datasets/ppi_mips.graphml'),'graphml')

## Define proximal accelerated gradient descent function

In [71]:
def proximal_accel_gradient_descent(A, d, ref_node, rho, alpha, eps, stepsize=1.):
    
    # Number of nodes in the graph
    n = d.shape[0]
    
    # Initialize seed vector
    seed = np.zeros(n)
    seed[ref_node] = 1
    
    # Initialized paramters
    x = np.zeros(n)

    # Initiliaze algorithm statistics
    err = 100.
    ite = 0
    
    # Another vector that we will need
    y = x
    
    # Compute gradient
    grad = ((1+alpha)/2)*(d*y)  - ((1-alpha)/2)*(A@y) - alpha * seed

    # The algorithm starts here
    while err > eps:
        
        # beta is a parameter for the accelerated algorithm
        if ite == 0:
            beta = 0
        else:
            beta = (1-np.sqrt(alpha)/(1+np.sqrt(alpha)))
            
        # Update parameters using a gradient step
        z = y - stepsize * grad / d 
        
        # Store old parameters
        x_old = x
        
        # Update parameters using the proximal step
        x = np.sign(z) * np.maximum(abs(z) - stepsize * rho * alpha, 0)
                
        # Update parameters y
        y = x + beta*(x - x_old)
        
        # Compute gradient
        grad = ((1+alpha)/2)*(d*y)  - ((1-alpha)/2)*(A@y) - alpha * seed
    
        # Compute termination criterion
        err = np.linalg.norm(x_old - x, np.inf)
        
        # Increase iteration count
        ite += 1
        
    return x

## Run proximal accelerated gradient descent on the givne graph

In [72]:
# L1-regularization parameter for the spectral problem
rho = 8.0e-5
# Teleportation parameter of the PageRank model
alpha = 0.1
# Refence node
ref_node = 1

# Matrices and vectors that are needed by the present implementation of proximal gradient descent
A = g.adjacency_matrix.tocsc()
d = g.d

# Stepsize for algorithm
stepsize = 2/(1+alpha)
# Termination tolerance
epsilon = 1.0e-10

start2 = time.time()
x = proximal_accel_gradient_descent(A, d, ref_node, rho, alpha, epsilon, stepsize = stepsize)
end2 = time.time()
print(end2 - start2)

0.0026030540466308594


## Print output

In [73]:
np.nonzero(x)[0]

array([ 1,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57])

In [74]:
x[np.nonzero(x)[0]]

array([1.62538022e-03, 1.50361708e-08, 1.52635292e-07, 2.92859895e-07,
       4.35785847e-07, 5.81491968e-07, 7.30060173e-07, 8.81575627e-07,
       1.03612691e-06, 1.19380619e-06, 1.35470939e-06, 1.51893641e-06,
       1.68659132e-06, 1.85778257e-06, 2.03262325e-06, 2.21123131e-06,
       2.39372985e-06, 2.58024736e-06, 2.77091811e-06, 2.96588235e-06,
       3.16528678e-06, 3.36928482e-06, 3.57803704e-06, 3.79171161e-06,
       4.01048471e-06, 4.23454102e-06, 4.46407426e-06, 4.69928774e-06,
       4.94039495e-06, 5.18762022e-06, 5.44119940e-06, 5.70138065e-06,
       5.96842519e-06, 6.24260823e-06, 6.52421986e-06, 6.81356615e-06,
       7.11097018e-06, 7.41677329e-06, 7.73133639e-06, 8.05504135e-06,
       8.38829256e-06, 8.73151864e-06, 9.08517424e-06, 9.44974207e-06,
       9.82573510e-06, 1.02136990e-05, 1.06142146e-05, 1.10279011e-05,
       1.14554191e-05, 1.18974741e-05, 1.23548202e-05, 1.28282649e-05])

## Define proximal gradient descent

In [75]:
def proximal_gradient_descent(A, d, ref_node, rho, alpha, eps, stepsize=1.):
    
    # Number of nodes in the graph
    n = d.shape[0]
    
    # Initialize seed vector
    seed = np.zeros(n)
    seed[ref_node] = 1
    
    # Initialized paramters
    x = np.zeros(n)

    # Initiliaze algorithm statistics
    err = 100.
    ite = 0

    # The algorithm starts here
    while err > eps:
        # Store old parameters
        x_old = x
        # Compute gradient
        grad = ((1+alpha)/2)*(d*x)  - ((1-alpha)/2)*(A@x) - alpha * seed
        # Update parameters using a gradient step
        x = x - stepsize * grad / d 
        # Update parameters using the proximal step
        x = np.sign(x) * np.maximum(abs(x) - stepsize * rho * alpha, 0)
        # Compute termination criterion
        err = np.linalg.norm(x_old - x, np.inf)
        # Increase iteration count
        ite += 1
        
    return x

## Run proximal gradient descent

In [76]:
# L1-regularization parameter for the spectral problem
rho = 8.0e-5
# Teleportation parameter of the PageRank model
alpha = 0.1
# Refence node
ref_node = 1

# Matrices and vectors that are needed by the present implementation of proximal gradient descent
A = g.adjacency_matrix
d = g.d

# Stepsize for algorithm
stepsize = 2/(1+alpha)
# Termination tolerance
epsilon = 1.0e-10

# Call proximal gradient descent
x = proximal_gradient_descent(A, d, ref_node, rho, alpha, epsilon, stepsize = stepsize)

In [77]:
np.nonzero(x)[0]

array([ 1,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57])

In [78]:
x[np.nonzero(x)[0]]

array([1.62537990e-03, 1.47025224e-08, 1.52298512e-07, 2.92519924e-07,
       4.35442625e-07, 5.81145430e-07, 7.29710256e-07, 8.81222265e-07,
       1.03577003e-06, 1.19344572e-06, 1.35434527e-06, 1.51856856e-06,
       1.68621966e-06, 1.85740702e-06, 2.03224373e-06, 2.21084773e-06,
       2.39334212e-06, 2.57985540e-06, 2.77052181e-06, 2.96548164e-06,
       3.16488154e-06, 3.36887494e-06, 3.57762243e-06, 3.79129216e-06,
       4.01006029e-06, 4.23411152e-06, 4.46363956e-06, 4.69884771e-06,
       4.93994945e-06, 5.18716912e-06, 5.44074257e-06, 5.70091792e-06,
       5.96795642e-06, 6.24213325e-06, 6.52373851e-06, 6.81307825e-06,
       7.11047556e-06, 7.41627176e-06, 7.73082775e-06, 8.05452539e-06,
       8.38776908e-06, 8.73098740e-06, 9.08463502e-06, 9.44919462e-06,
       9.82517917e-06, 1.02131343e-05, 1.06136409e-05, 1.10273181e-05,
       1.14548265e-05, 1.18968715e-05, 1.23542073e-05, 1.28276414e-05])